In [ ]:
#| default_exp helper.latex.augment

# helper.latex.augment
> Latex functions for modifying latex text, mainly for data augmentation

In [ ]:
#| export
import random
import re
import string
from typing import Callable, List, Optional, Union


In [ ]:
from fastcore.test import *

## Augment latex text

For data augmentation, it can be useful to introduce latex typos intentionally. The following functions do so.

### Modify just latex str

In [ ]:
#| export

FONT_STYLE_COMMANDS = [
    "mathscr",
    "mathcal",
    "mathfrak",
    "mathbb",
    "mathbf",
    "mathrm",
    "operatorname",
    "text",
    ]
UNCOMMON_FONT_STYLE_COMMANDS = [
    "mathit",
    "mathsf",
    "mathtt",
]
# COMMON_FONT_STYLE_TYPOS = {
#     "mathscr": {"mathcal", "mathfrak"},
#     "mathcal": {"mathscr"},
#     "mathrm": {"operatorname"},
#     "mathrmfrak": {"mathcal", "mathbf"}
# }


def modify_at_random(
        latex_string: str, # A latex str, surrounded by dollar signs (either single or double) as necessary.
        pattern: Union[str,re.Pattern],
        chance: float, # The chance that each change is performed
        replace_func: Callable[[re.Match, float], str],
        seed: Optional[int] = None
    ) -> str:
    # Set the random seed if provided
    if seed is not None:
        random.seed(seed)
    result = re.sub(pattern, lambda x: replace_func(x, chance), latex_string)
    return result



In [ ]:
# TODO: test

In [ ]:
#| export
def remove_font_styles_at_random(
        latex_string: str, # A latex str, surrounded by dollar signs (either single or double) as necessary.
        p: float = 0.05, # The chance that each font styling comand is removed
        seed: Optional[int] = None
        ) -> str: 
    """Randomly remove font style commands at random from `latex_string`.
    """
    # Combine all font style commands
    all_commands = FONT_STYLE_COMMANDS # + UNCOMMON_FONT_STYLE_COMMANDS
    # Create a regex pattern to match all font style commands
    pattern = r'\\(' + '|'.join(all_commands) + r')\s*\{([^{}]*(?:\{[^{}]*\}[^{}]*)*)\}'
    def replace_func(match, p: float):
        # Randomly decide whether to remove the command
        if random.random() < p:
            # If removed, return only the content inside the braces
            return match.group(2)
        else:
            # If not removed, return the original match
            return match.group(0)
    return modify_at_random(latex_string, pattern, p, replace_func, seed)

In [ ]:
# Test 1: Basic removal
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic}$"
result = remove_font_styles_at_random(latex, p=1.0, seed=42)
assert result == "$Bold and Calligraphic$"

# Test 2: No removal
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic}$"
result = remove_font_styles_at_random(latex, p=0.0, seed=42)
assert result == latex

# Test 3: Partial removal
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic} and \mathfrak{Fraktur}$"
result = remove_font_styles_at_random(latex, p=0.5, seed=42)
assert result != latex
assert result != "$Bold and Calligraphic and Fraktur$"

# Test 4: Nested commands
# latex = r"$\mathbf{\mathcal{Nested}}$"
# result = remove_font_styles_at_random(latex, chance=1.0, seed=42)
# assert result == "$Nested$"

# Test 5: Uncommon commands
# latex = r"$\mathtt{Typewriter} and \mathsf{Sans Serif}$"
# result = remove_font_styles_at_random(latex, chance=1.0, seed=42)
# assert result == "$Typewriter and Sans Serif$"

# Test 6: Text and operatorname
latex = r"$\text{Plain text} and \operatorname{sin}(x)$"
result = remove_font_styles_at_random(latex, p=1.0, seed=42)
assert result == "$Plain text and sin(x)$"

# Test 7: No commands present
latex = "$x + y = z$"
result = remove_font_styles_at_random(latex, p=1.0, seed=42)
assert result == latex

# Test 8: Multiple dollar signs
latex = r"$$\mathbf{Equation}: E = mc^2$$"
result = remove_font_styles_at_random(latex, p=1.0, seed=42)
assert result == "$$Equation: E = mc^2$$"

# Test 9: Seed consistency
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic}$"
result1 = remove_font_styles_at_random(latex, p=0.5, seed=42)
result2 = remove_font_styles_at_random(latex, p=0.5, seed=42)
assert result1 == result2

# Test 10: Different seeds
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic}$"
result1 = remove_font_styles_at_random(latex, p=0.5, seed=42)
result2 = remove_font_styles_at_random(latex, p=0.5, seed=43)
assert result1 != result2


In [ ]:
#| export

def change_font_styles_at_random(
        latex_string: str,
        p: float = 0.1,
        seed: Optional[int] = None
        ) -> str:
    """Randomly change font style commands in `latex_string`."""
    all_commands = FONT_STYLE_COMMANDS # + UNCOMMON_FONT_STYLE_COMMANDS
    pattern = r'\\(' + '|'.join(all_commands) + r')\s*\{([^{}]*(?:\{[^{}]*\}[^{}]*)*)\}'
    def replace_func(match, p: float):
            if random.random() < p:
                current_command = match.group(1)
                new_command = random.choice([cmd for cmd in all_commands if cmd != current_command])
                return f"\\{new_command}{{{match.group(2)}}}"
            return match.group(0)
    return modify_at_random(latex_string, pattern, p, replace_func, seed)

In [ ]:

# Test 1: Basic functionality
latex = r"$\mathbf{Bold}$"
result = change_font_styles_at_random(latex, p=1.0, seed=42)
# print(result)
test_ne(result, latex)
test_eq(re.match(r"\$\\\w+{Bold}\$", result) is not None, True)


# Test 2: No change with chance 0
latex = r"$\mathcal{Calligraphic}$"
result = change_font_styles_at_random(latex, p=0.0, seed=42)
test_eq(result, latex)

# Test 3: Multiple commands
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic}$"
result = change_font_styles_at_random(latex, p=1.0, seed=42)
test_ne(result, latex)
test_eq(re.match(r"\$\\\w+{Bold} and \\\w+{Calligraphic}\$", result) is not None, True)

# Test 4: Nested commands
latex = r"$\mathbf{\mathcal{Nested}}$"
result = change_font_styles_at_random(latex, p=1.0, seed=42)
test_ne(result, latex)
test_eq(re.match(r"\$\\\w+{\\\w+{Nested}}\$", result) is not None, True)

# Test 5: Uncommon commands
# latex = r"$\mathtt{Typewriter}$"
# result = change_font_styles_at_random(latex, p=1.0, seed=42)
# test_ne(result, latex)
# test_eq(re.match(r"\$\\\w+{Typewriter}\$", result) is not None, True)

# Test 6: Consistency with same seed
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic}$"
result1 = change_font_styles_at_random(latex, p=0.5, seed=42)
result2 = change_font_styles_at_random(latex, p=0.5, seed=42)
test_eq(result1, result2)

# Test 7: Different results with different seeds
latex = r"$\mathbf{Bold} and \mathcal{Calligraphic}$"
result1 = change_font_styles_at_random(latex, p=0.5, seed=42)
result2 = change_font_styles_at_random(latex, p=0.5, seed=43)
test_ne(result1, result2)


In [ ]:
#| export
# List of Greek letters in LaTeX
GREEK_LETTERS = [
    'alpha', 'beta', 'gamma', 'delta', 'epsilon', 'zeta', 'eta', 'theta', 'iota', 'kappa', 'lambda', 'mu', 
    'nu', 'xi', 'omicron', 'pi', 'rho', 'sigma', 'tau', 'upsilon', 'phi', 'chi', 'psi', 'omega',
    'Gamma', 'Delta', 'Theta', 'Lambda', 'Xi', 'Pi', 'Sigma', 'Upsilon', 'Phi', 'Psi', 'Omega'
]

def change_greek_letters_at_random(
        latex_string: str,
        p: float = 0.05
) -> str:
    """Randomly change Greek letters in `latex_string`."""
    def replace_func(match, p: float):
        if random.random() < p:
            current_letter = match.group(1)
            new_letter = random.choice([l for l in GREEK_LETTERS if l != current_letter])
            return f"\\{new_letter}"
        return match.group(0)
    # Pattern to match Greek letters
    pattern = r'\\(' + '|'.join(GREEK_LETTERS) + r')\b'
    # Apply the replacement
    result = re.sub(pattern, lambda x: replace_func(x, p), latex_string)
    return result


In [ ]:
# Test 1: Basic functionality
latex = r"$\alpha + \beta = \gamma$"
result = change_greek_letters_at_random(latex, p=1.0)
test_ne(result, latex)
# test(all(letter in result for letter in [r'\alpha', r'\beta', r'\gamma']))

# Test 2: No change with chance 0
latex = r"$\delta \times \epsilon$"
result = change_greek_letters_at_random(latex, p=0.0)
test_eq(result, latex)

# Test 3: Mixed content
latex = r"$f(x) = \theta x + \phi$"
result = change_greek_letters_at_random(latex, p=1.0)
test_ne(result, latex)
# test(all(letter in result for letter in [r'\theta', r'\phi']))
# test('f(x) =' in result)

# Test 4: Uppercase Greek letters
latex = r"$\Gamma(x) + \Delta y = \Omega$"
result = change_greek_letters_at_random(latex, p=1.0)
test_ne(result, latex)
# test(all(letter in result for letter in [r'\Gamma', r'\Delta', r'\Omega']))

# Test 5: Multiple occurrences
latex = r"$\alpha + \alpha = 2\alpha$"
result = change_greek_letters_at_random(latex, p=1.0)
test_ne(result, latex)
# test(result.count('\\') == 3)  # Ensure all Greek letters were changed


In [ ]:
#| export
# def push_dollar_signs_surrounding_latex(
#         latex_string: str, # A latex str, surrounded by dollar signs (either single or double) as necessary.
#         remove_pushed_out_font_style_command: bool = True
#         ) -> str:
#     """
#     Modify `latex_string` so that in effect, dollar signs are 
#     """
#     return ""

### Modify latex str 

In [ ]:
#| export
def random_char_modification(text, p=0.05):
    """
    Randomly change characters in `text`.
    """
    chars = list(text)
    all_chars = string.ascii_letters + string.digits + string.punctuation + ' '
    for i in range(len(chars)):
        if random.random() < p:
            action = random.choice(['delete', 'add', 'modify'])
            if action == 'delete':
                chars[i] = ''
            elif action == 'add':
                chars.insert(i, random.choice(all_chars))
            else:
                chars[i] = random.choice(all_chars)
    return ''.join(chars)

In [ ]:
# Test 1: Verify p=0 leaves text unchanged (Identity)
test_eq(random_char_modification("Test String", p=0), "Test String")

# Test 2: Verify empty string returns empty string
test_eq(random_char_modification("", p=0.5), "")

# Test 3: Verify output is always a string
res = random_char_modification("Type Check", p=0.5)
assert isinstance(res, str)

# Test 4: Verify random behavior with a seed (Reproducibility)
# Setting a seed ensures the 'random' choices are the same every time we run the test
random.seed(42)
input_text = "hello world"
# With p=1.0, changes are guaranteed. We check against the expected deterministic output for this seed.
# Based on the logic: 'h' -> modified/added/deleted...
modified = random_char_modification(input_text, p=1.0)

test_ne(modified, input_text) # Should not be equal
# Optional: exact match test for the seed if you want strict regression testing
test_eq(modified, "nld@Cald") 

In [ ]:
#| export
# def dollar_sign_manipulation(text, p=0.05):
#     """
#     Either delete or move dollar signs (which are usually there for latex math mode) from `text`,
#     while preserving all whitespace characters.
#     """
#     # Split the text into tokens, preserving whitespace
#     tokens = re.split(r'(\s+)', text)
    
#     # Find indices of non-whitespace tokens containing '$'
#     dollar_indices = [i for i, token in enumerate(tokens) if '$' in token and not token.isspace()]
    
#     for i in dollar_indices:
#         if random.random() < p:
#             action = random.choice(['delete', 'move'])
#             if action == 'delete':
#                 tokens[i] = tokens[i].replace('$', '')
#             else:
#                 if len(dollar_indices) > 1:
#                     # Find a new position for the dollar sign
#                     possible_positions = [pos for pos in dollar_indices if pos != i]
#                     new_pos = random.choice(possible_positions)
                    
#                     # Move the dollar sign
#                     tokens[new_pos] += '$'
#                     tokens[i] = tokens[i].replace('$', '')
#                 else:
#                     # If there's only one dollar sign, we can't move it, so we'll delete it instead
#                     tokens[i] = tokens[i].replace('$', '')

#     return ''.join(tokens)


In [ ]:
#| export

def dollar_sign_manipulation(text, p=0.05):
    """
    Either delete or move dollar signs from `text` while preserving whitespace.
    """
    tokens = re.split(r'(\s+)', text)
    # Tokens with '$' are inherently not whitespace, so we can skip the .isspace() check
    d_idxs = [i for i, t in enumerate(tokens) if '$' in t]
    
    for i in d_idxs:
        if random.random() < p:
            # Step 1: Always remove '$' from the current token (happens in both 'delete' and 'move')
            tokens[i] = tokens[i].replace('$', '')
            
            # Step 2: If we are "moving" (50% chance) and have a valid target, append '$' there
            # This replicates the original logic: Move = Remove from A + Add to B
            if len(d_idxs) > 1 and random.choice([True, False]):
                target = random.choice([j for j in d_idxs if j != i])
                tokens[target] += '$'

    return ''.join(tokens)


In [ ]:

# Test 1: Verify p=0 leaves text unchanged (Identity)
txt = " The price is $5.00 "
test_eq(dollar_sign_manipulation(txt, p=0), txt)

# Test 2: Verify text without dollar signs is unaffected
no_dollar_txt = "Hello world"
test_eq(dollar_sign_manipulation(no_dollar_txt, p=1.0), no_dollar_txt)

# Test 3: Verify single dollar sign behavior (p=1.0)
# Logic check: If there is only 1 dollar sign, 'move' is impossible.
# The code defaults to 'delete' in the else block.
# So "Price $5" -> "Price 5" every time if p=1.0.
single_dollar = "Price $5"
test_eq(dollar_sign_manipulation(single_dollar, p=1.0), "Price 5")

# Test 4: Verify whitespace is preserved exactly
# Even if operations happen, the split regex r'(\s+)' ensures spaces are kept.
ws_input = "  $x  $y  "
# We don't check the exact $ positions here, just that the spaces are still there
res_ws = dollar_sign_manipulation(ws_input, p=1.0)
assert res_ws.startswith("  ")
assert "  " in res_ws[2:].strip() # check middle spaces exist
assert res_ws.endswith("  ")

# Test 5: Verify complex random behavior with seed (Reproducibility)
random.seed(42)
input_text = "Let $x$ be $y$"
# With p=1.0 and seed 42:
# 1. First '$x$': random < p, choice might be delete or move.
# 2. Second '$y$': random < p...
# Based on execution, this seed results in 'Let x be y' (both got deleted or moved into oblivion)
test_eq(dollar_sign_manipulation(input_text, p=1.0), "Let x$ be y")


In [ ]:
#| export

def remove_math_keywords(text, p=0.05):
    """
    Remove all mentions of Definition/Theorem/Remark, etc.
    """
    keywords = r"(Definition|Remark|Proposition|Exercise|Example|Theorem|Lemma|Corollary)\s+\w+(\.\w+){1,3}"
    def random_remove(match):
        if random.random() < p:
            return ''
        else:
            return match.group(0)
    return re.sub(keywords, random_remove, text)


In [ ]:
#| export

def random_word_removal(text, p=0.05):
    """
    Randomly remove words while preserving all whitespace characters.
    """
    # Split the text into tokens, preserving whitespace
    tokens = re.split(r'(\s+)', text)
    
    # Process non-whitespace tokens
    result = []
    for token in tokens:
        if token.strip():  # If the token is not just whitespace
            if random.random() > p:
                result.append(token)
        else:
            result.append(token)  # Always keep whitespace tokens
    
    return ''.join(result)


In [ ]:
#| export
def random_latex_command_removal(text, p=0.1):
    """
    Randomly remove latex commands
    """
    return re.sub(r'\\[a-zA-Z]+(\{[^}]*\})?', lambda m: m.group(0) if random.random() > p else '', text)

In [ ]:
#| export
# TODO: this function, as implemented, is very buggy
def push_dollar_signs(
        latex: str,
        p: float = 0.1, # Push probability
        seed: int = None,
        return_indices_of_math_mode_content: bool = False # If `True`, additionally return a `list[tuple[int, int]]` of indices within the outputted `str` signifiying the location of what was essentially the content of the original math mode.
        ) -> tuple[str, list[tuple[int, int]]]:
    """
    Push dollar signs delimiting math mode into each other at random within a text.
    """
    
    if seed is not None:
        random.seed(seed)

    font_commands = FONT_STYLE_COMMANDS + UNCOMMON_FONT_STYLE_COMMANDS 
    font_commands = [rf'\{font_command}' for font_command in font_commands]
    
    # Find all single and double dollar sign positions
    dollar_positions = [(m.start(), m.end()) for m in re.finditer(r'\${1,2}', latex)]
    
    # Ensure we have an even number of dollar sign groups
    if len(dollar_positions) % 2 != 0:
        dollar_positions = dollar_positions[:-1]

    new_indices_of_math_mode_content: list[tuple[int, int]] = []
    for i in range(0, len(dollar_positions), 2):
        start, end = dollar_positions[i], dollar_positions[i+1]
        if random.random() >= p:
            continue
        # if random.random() < push_probability:
            # Decide which dollar sign group to push
        push_start = random.choice([True, False])
        
        if push_start:
            new_start = push_dollar_sign(latex, start[1], direction='right')
            dollar_length = start[1] - start[0]
            new_starting_part = latex[:start[0]]
            pushed_out_math_mode_part = remove_split_commands(latex[start[1]:new_start], font_commands).rstrip() + ' ' + '$'*dollar_length
            new_ending_part = latex[new_start:].lstrip()
            latex = new_starting_part + pushed_out_math_mode_part + new_ending_part 
            # latex = latex[:start[0]] + pushed_out_math_mode_part + latex[new_start:].lstrip()
            new_indices_of_math_mode_content.append(
                (start[0], start[0] + len(pushed_out_math_mode_part) + new_ending_part.index('$'*dollar_length) + dollar_length))
        else:
            new_end = push_dollar_sign(latex, end[0], direction='left')
            dollar_length = end[1] - end[0]
            new_starting_part = latex[:new_end].rstrip()
            pushed_out_math_mode_part = '$'*dollar_length + ' ' + remove_split_commands(latex[new_end:end[0]], font_commands).lstrip()
            new_ending_part = latex[end[1]:]
            latex = new_starting_part + pushed_out_math_mode_part + new_ending_part
            last_dollar_in_new_starting_part = new_starting_part.rindex('$'*dollar_length)
            new_indices_of_math_mode_content.append(
                # (start[0], start[0] + len(pushed_out_math_mode_part))
                (last_dollar_in_new_starting_part, len(new_starting_part) + len(pushed_out_math_mode_part))
                )
    
        # Update dollar positions for the next pair
        dollar_positions = [(m.start(), m.end()) for m in re.finditer(r'\${1,2}', latex)]

    if return_indices_of_math_mode_content:
        return latex, new_indices_of_math_mode_content
    else:
        return latex

def push_dollar_sign(latex: str, pos: int, direction: str) -> int:
    """Push the dollar sign in the specified direction to the next word boundary."""
    if direction == 'right':
        next_space = latex.find(' ', pos)
        if next_space == -1:
            return len(latex)
        next_non_space = next_space
        while next_non_space < len(latex) and latex[next_non_space].isspace():
            next_non_space += 1
        return next_non_space
    else:  # left
        prev_space = latex.rfind(' ', 0, pos)
        if prev_space == -1:
            return 0
        prev_non_space = prev_space
        while prev_non_space > 0 and latex[prev_non_space-1].isspace():
            prev_non_space -= 1
        return prev_non_space

def remove_split_commands(latex: str, commands: List[str]) -> str:
    """Remove font style commands from the latex string."""
    for cmd in commands:
        pattern = re.escape(cmd) + r'\s*\{([^}]*)\}'
        latex = re.sub(pattern, r'\1', latex)
    return latex

In [ ]:
# Test the function
latex1 = r"This is $\mathbf{bold} and \mathrm{roman}$ text"
latex2 = r"This is $$\mathbf{bold} and \mathrm{roman}$$ text"
result1 = push_dollar_signs(latex1, p=0.7, seed=17)
result2 = push_dollar_signs(latex2, p=0.7, seed=17)
print(result1)
print(result2)

This is $\mathbf{bold} and$ roman text
This is $$\mathbf{bold} and$$ roman text


In [ ]:

latex = r"This is $\mathbf{bold} and \mathrm{roman}$ text"
result = push_dollar_signs(latex, p=1.0, seed=42)
print(result)
assert r'\mathbf' not in result
assert r'\mathrm' in result

latex = r"This is $\mathbf{bold} and \mathrm{roman}$ text"
result = push_dollar_signs(latex, p=1.0, seed=17)
print(result)
assert r'\mathrm' not in result
assert r'\mathbf' in result

This is bold $and \mathrm{roman}$ text
This is $\mathbf{bold} and$ roman text


In [ ]:
latex = r"This is $\mathbf{bold}_a^b and \mathrm{roman}$ text"
result = push_dollar_signs(latex, p=1.0, seed=42)
print(result)

This is bold_a^b $and \mathrm{roman}$ text


In [ ]:
latex = r"This is $\mathbf{bold}and \mathrm{roman}$ text"
result = push_dollar_signs(latex, p=1.0, seed=42)
print(result)

This is boldand $\mathrm{roman}$ text


In [ ]:
latex = r"This is $\mathbf{bold}_a^b and \mathrm{roman}$ text; This is $$\mathbf{bold}_a^b and \mathrm{roman}$$ text"
result, original_math_mode_content_indices = push_dollar_signs(latex, p=1.0, seed=17, return_indices_of_math_mode_content=True)
print(result)
print(original_math_mode_content_indices)
print(result[original_math_mode_content_indices[0][0]:original_math_mode_content_indices[0][1]])
print(result[original_math_mode_content_indices[1][0]:original_math_mode_content_indices[1][1]])

This is $\mathbf{bold}_a^b and$ roman text; This is $$\mathbf{bold}_a^b and$$ roman text
[(8, 37), (52, 83)]
$\mathbf{bold}_a^b and$ roman
$$\mathbf{bold}_a^b and$$ roman


In [ ]:
print(result[52:126])

$$\mathbf{bold}_a^b and$$ roman text


In [ ]:
latex = r"This is $A + B=C$"
result = push_dollar_signs(latex, p=1.0, seed=43)
print(result)

This is A $+ B=C$


In [ ]:
latex = r"This is $\mathbf{bold} and \mathrm{roman}$ text"
result, original_math_mode_content_indices = push_dollar_signs(latex, p=1.0, seed=42, return_indices_of_math_mode_content=True)
print(result)
print(original_math_mode_content_indices)
print(result[original_math_mode_content_indices[0][0]:original_math_mode_content_indices[0][1]])

This is bold $and \mathrm{roman}$ text
[(8, 33)]
bold $and \mathrm{roman}$


In [ ]:
latex = r"This is $\mathbf{bold} and \mathrm{roman}$ text"
result, original_math_mode_content_indices = push_dollar_signs(latex, p=1.0, seed=17, return_indices_of_math_mode_content=True)
print(result)
print(original_math_mode_content_indices)
print(result[original_math_mode_content_indices[0][0]:original_math_mode_content_indices[0][1]])

This is $\mathbf{bold} and$ roman text
[(8, 33)]
$\mathbf{bold} and$ roman


In [ ]:
#| export

### Using the modification functions to augment text

In [ ]:
#| export
def augment_text(
        text: str,
        methods: list[Callable[[str], str]],
        ) -> str:
    """
    Augment `text` by applying modification methods.
    """
    for method in methods:
        text = method(text)
    return text

In [ ]:
#| export
# def add_typos(
#         latex_string: str, # A latex str, surrounded by dollar signs (either single or double) as necessary.
#         seed: Optional[int] = None
#         ) -> str: # A new str that is a modification of `latex_string` with "typos".

#     return ""



In [ ]:
#| export
def _create_method(method, p, scale):
    """
    Helper function to `choose_modification_methods_at_random`
    """
    return lambda x: method(x, p=p*scale)

In [ ]:
#| export
def choose_modification_methods_at_random(
        methods: list[tuple[Callable, float]],
        method_inclusion_chance = float, # The chance to include each method
        scale = float, # The amount by which to "scale" the method's tendency to modify the text.
        ) -> list[Callable[[str], str]]:
    random_methods: list[Callable[[str], str]] = []
    for method, p in methods:
        if random.random() < method_inclusion_chance:
            random_methods.append(_create_method(method, p, scale))
    return random_methods